# 5.1 pandas的数据结构介绍--------------------------------------------------------------------

# Series

Series是一种类似于一维数组的对象，它由**一组数据**（各种NumPy数据类型）以及一组与之相关的**数据标签**（即索引）组成。

Series的字符串表现形式为：索引在左边，值在右边。由于我们没有为数据指定索引，于是会自动创建一个0到N-1（N为数据的长度）的整数型索引。
你可以通过**Series 的values和index属性**获取其数组表示形式和索引对象

In [1]:
import pandas as pd
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [2]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

与普通NumPy数组相比，你可以通过索引的方式选取Series中的单个或一组值：**【取值时索引可重复】**

In [3]:
obj2[['c','a','c']]

c    3
a   -5
c    3
dtype: int64

使用NumPy函数或类似NumPy的运算（如根据布尔型数组进行过滤、标量乘法、应用数学函数等）**都会保留索引值的链接**：

还可以将Series看成是一个定长的有序字典，因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

In [4]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

如果只传入一个字典，则结果Series中的索引就是原字典的键（**有序排列**）。你可以传入排好序的字典的键以改变顺序：

In [5]:
states = ['California', 'Ohio', 'Oregon', 'Texas',]
obj4 = pd.Series(sdata, states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

在这个例子中，sdata中跟states索引相匹配的那3个值会被找出来并放到相应的位置上，但由于"California"所对应的sdata值找不到，所以其结果就为NaN（即“非数字”（not a number），在pandas中，它用于表示缺失或NA值）。因为‘Utah’不在states中，它被从结果中除去。

**【说明series的创建以index为准绳】**

我将使用缺失（missing）或NA表示缺失数据。pandas的isnull和notnull函数可用于检测缺失数据：

In [6]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

Series也有类似的实例方法：

In [7]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

对于许多应用而言，Series最重要的一个功能是，它会根据运算的索引标签自动对齐数据：

数据对齐功能将在后面详细讲解。如果你使用过数据库，你可以认为是类似join的操作。

In [8]:
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

In [9]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [10]:
obj3+obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series**对象本身**及**其索引**都有一个name属性，该属性跟pandas其他的关键功能关系非常密切：

In [11]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

Series的索引可以通过赋值的方式就地修改：

In [12]:
obj = pd.Series([4,7,-5,3])

In [13]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

# DataFrame

DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔值等）。DataFrame既有行索引也有列索引；
它可以被**看做由Series组成的字典（共用同一个索引）**。
DataFrame中的数据是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）。

建DataFrame的办法有很多，最常用的一种是直接传入一个由**等长列表或NumPy数组组成的字典**：

In [14]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002
5,3.2,Nevada,2003


对于特别大的DataFrame，head方法会选取前五行：

如果指定了**列序列**，则DataFrame的列就会按照指定顺序进行排列：

In [15]:
pd.DataFrame(data,columns=['year','state','pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


如果传入的列在数据中找不到，就会在结果中产生缺失值：**【但是index不能超出，否则报错】**

In [16]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


通过类似字典标记的方式或属性的方式，可以将**DataFrame的列获取**为一个Series：

注意，返回的Series拥有原DataFrame相同的索引，且其name属性也已经被相应地设置好了。

In [17]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object


笔记：IPython提供了类似属性的访问（即frame2.year）和tab补全。
**frame2[column]**适用于任何列的名，但是**frame2.column**只有在列名是一个合理的Python变量名时才适用。

行也可以通过位置或名称的方式进行获取，比如用loc属性（稍后将对此进行详细讲解）：

In [18]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式进行修改。例如，我们可以给那个空的"debt"列赋上一个标量值或一组值：

将列表或数组赋值给某个列时，其长度必须跟DataFrame的长度相匹配。
**如果赋值的是一个Series**，就会精确匹配DataFrame的索引，所有的空位都将被填上缺失值：

In [19]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


为不存在的列赋值会创建出一个新列。关键字del用于删除列。

作为del的例子，我先添加一个新的布尔值的列，state是否为'Ohio'：

注意：不能用frame2.eastern创建新的列。

In [20]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


In [21]:
del frame2['eastern']
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


注意：通过索引方式返回的列只是相应数据的视图而已，并不是副本。因此，对返回的Series所做的任何就地修改全都会反映到源DataFrame上。通过Series的copy方法即可指定复制列。

另一种常见的数据形式是**嵌套字典**：

如果嵌套字典传给DataFrame，pandas就会被解释为：外层字典的键作为列，内层键则作为行索引：

In [22]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


你也可以使用类似NumPy数组的方法，对DataFrame进行**转置**（交换行和列）：

In [23]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


内层字典的键会被合并、排序以形成最终的索引。如果明确指定了索引，则不会这样：

In [24]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


由Series组成的字典差不多也是一样的用法：

In [25]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7


> 表5-1列出了DataFrame构造函数所能接受的各种数据。

> 二维ndarray | 数组、列表、元组3构成的字典 | numpy的结构化数组 | Series组成的字典 | 嵌套字典 | 字典或Series的列表 |

> 列表或元组组成的列表 | 另一个DataFrame | Numpy的MaskedArray |

如果设置了DataFrame的index和columns的name属性，则这些信息也会被显示出来：

In [26]:
frame3.index.name = 'year' ; frame3.columns.name = 'state' ; frame3.name = 'xxx'
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [27]:
frame3.index

Int64Index([2000, 2001, 2002], dtype='int64', name='year')

跟Series一样，**values属性**也会以二维ndarray的形式返回DataFrame中的数据：

In [28]:
frame3.values

array([[ nan,  1.5],
       [ 2.4,  1.7],
       [ 2.9,  3.6]])

如果DataFrame各列的数据类型不同，则值数组的dtype就会选用能兼容所有列的数据类型：

In [29]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

# 索引对象 Index Objects

pandas的索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建Series或DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个Index：

**Index对象是不可变**的，因此用户不能对其进行修改：

不可变可以使Index对象在多个数据结构之间安全共享：

In [30]:
import numpy as np
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [31]:
obj2 = pd.Series([1.5,-2.5,0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

除了类似于数组，Index的功能也类似一个固定大小的集合：

与python的集合不同，pandas的Index可以包含重复的标签：选择重复的标签，会显示所有的结果。

In [32]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [33]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [34]:
'Ohio' in frame3.columns

True

> 每个索引都有一些方法和属性，它们可用于设置逻辑并回答有关该索引所包含的数据的常见问题。表5-2列出了这些函数。

> append | difference intersection union | isin | delete drop insert | is_monotonic is_unique unique |

# 5.2 基本功能--------------------------------------------------------------------------------------------

本节中，我将介绍操作Series和DataFrame中的数据的基本手段。后续章节将更加深入地挖掘pandas在数据分析和处理方面的功能。本书不是pandas库的详尽文档，主要关注的是最重要的功能

# 重塑索引

pandas对象的一个重要方法是reindex，其作用是创建一个新对象，它的数据符合新的索引。看下面的例子：

In [35]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

用该Series的**reindex方法**将会根据新索引进行重排。如果某个索引值当前不存在，就引入缺失值：

In [36]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。**method选项**即可达到此目的，例如，使用ffill可以实现前向值填充：

In [37]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [38]:
obj3.reindex(range(6),method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

借助DataFrame，reindex可以修改（行）索引和列。只传递一个序列时，会重新索引结果的行：

In [39]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [40]:
frame2 = frame.reindex(['a','b','c','d'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


列可以用**columns关键字**重新索引：

In [41]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


> 表5-3列出了reindex函数的各参数及说明。

> index | method | fill_value | limit最大填充量 | tolerance最大间距 | level | copy |

# 丢弃指定轴上的项

丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。

由于需要执行一些数据整理和集合逻辑，所以**drop方法**返回的是一个在指定轴上删除了指定值的**新对象**：

对于DataFrame，可以删除任意轴上的索引值。为了演示，先新建一个DataFrame例子：

In [42]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


用标签序列调用drop会从行标签（axis 0）删除值：

In [43]:
data.drop(['Colorado','Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


通过**axis参数**，传递axis=1或axis='columns'可以删除列的值：

In [44]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


通过**inplace参数**，许多函数，如drop，会修改Series或DataFrame的大小或形状，可以就地修改对象，不会返回新的对象：

小心使用inplace，它会销毁所有被删除的数据。

In [45]:
obj.drop('c', inplace=True)
obj

d    4.5
b    7.2
a   -5.3
dtype: float64

# 索引、选取和过滤

Series索引（obj[...]）的工作方式类似于NumPy数组的索引，只不过Series的索引值不只是整数。下面是几个例子：

利用标签的切片运算与普通的Python切片运算不同，其末端是包含的：【Series利用**标签的切片运算末端包含**】

用切片可以对Series的相应部分进行设置：

In [46]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [47]:
obj[['b','c']]

b    1.0
c    2.0
dtype: float64

用一个值或序列对DataFrame进行索引其实就是获取一个或多个列：

In [48]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
   .....:                     index=['Ohio', 'Colorado', 'Utah', 'New York'],
   .....:                     columns=['one', 'two', 'three', 'four'])

In [49]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [50]:
data[['three','one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


这种索引方式有几个特殊的情况。首先通过切片或布尔型数组选取数据：

选取行的语法**data[:2]**十分方便。向[ ]传递单一的元素或列表，就可选择列。

In [51]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


另一种用法是data[]内使用**布尔型DataFrame**（比如下面这个由标量比较运算得出的）进行索引：

这使得DataFrame的语法与NumPy二维数组的语法很像。

In [52]:
data<5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [53]:
data[data<5]

,one,two,three,four
Ohio,0.0,1.0,2.0,3.0
Colorado,4.0,NaN,NaN,NaN
Utah,NaN,NaN,NaN,NaN
New York,NaN,NaN,NaN,NaN


In [54]:
data[data<5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### 用loc和iloc进行选取

对于DataFrame的行的标签索引label-indexing on the rows，我引入了特殊的标签运算符loc和iloc。它们可以让你用类似NumPy的标记，

使用轴标签（loc）或整数索引（iloc），从DataFrame选择行和列的子集。

In [55]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

In [56]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

这两个索引函数也适用于一个标签或多个标签的切片：

In [57]:
data.loc[:'Utah', 'two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [58]:
data.iloc[:, :3]

,one,two,three
Ohio,0,0,0
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


In [59]:
data.iloc[:, :3][data.three>5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


> 在pandas中，有多个方法可以选取和重新组合数据。对于DataFrame，表5-4进行了总结。后面会看到，还有更多的方法进行层级化索引。

| df[val]              | Select single column or sequence of columns from the DataFrame; special case
conveniences: boolean array (filter rows), slice (slice rows), or boolean DataFrame
(set values based on some criterion)

| df.loc[val]            | Selects single row or subset of rows from the DataFrame by label

| df.loc[:, val]           | Selects single column or subset of columns by label

| df.loc[val1, val2]        |  Select both rows and columns by label

| df.iloc[where]           | Selects single row or subset of rows from the DataFrame by integer position

| df.iloc[:, where]         | Selects single column or subset of columns by integer position

| df.iloc[where_i, where_j]   | Select both rows and columns by integer position

| df.at[label_i, label_j]     | Select a single scalar value by row and column label

| df.iat[i, j]            | Select a single scalar value by row and column position (integers)

| reindex method           | Select either rows or columns by labels

| get_value, set_value methods | Select single value by row and column label

# 整数索引

为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确，请使用loc（标签）或iloc（整数）：

# 算术运算和数据对齐

pandas最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。

对于有数据库经验的用户，这就像在索引标签上进行自动外连接。

自动的数据对齐操作在**不重叠的索引**处引入了NA值。缺失值会在算术运算过程中传播。

In [60]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],index=['a', 'c', 'e', 'f', 'g'])
s1+s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

对于DataFrame，对齐操作会同时发生在行和列上：

把它们相加后将会返回一个新的DataFrame，其索引和列为原来那两个DataFrame的并集：

因为'c'和'e'列均不在两个DataFrame对象中，在结果中以缺省值呈现。行也是同样。

In [61]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12).reshape((4,3)), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [62]:
df2

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [63]:
df1+df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


In [64]:
df1-df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,-3.0,NaN,-2.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,-3.0,NaN,-2.0,NaN
Utah,NaN,NaN,NaN,NaN


### 在算术方法中填充值

在对不同索引的对象进行算术运算时，你可能希望当一个对象中某个轴标签在另一个对象中找不到时填充一个特殊值（比如0）：

In [65]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
   .....:                    columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
   .....:                    columns=list('abcde'))

In [66]:
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [67]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [68]:
df1+df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


使用df1的add方法，传入df2以及一个**fill_value参数**：

In [69]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


> 表5-5列出了Series和DataFrame的算术方法。它们每个都有一个副本，以字母r开头，它会翻转参数。

> add, radd | sub rsub | div rdiv | floordive rfloordiv | mul rmul | pow rpow |

### DataFrame和Series之间的运算

跟不同维度的NumPy数组一样，DataFrame和Series之间算术运算也是有明确规定的。

先来看一个具有启发性的例子，计算一个二维数组与其某行之间的差：

In [70]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [71]:
arr-arr[0]

array([[0, 0, 0, 0],
       [4, 4, 4, 4],
       [8, 8, 8, 8]])

当我们从arr减去arr[0]，每一行都会执行这个操作。这就叫做**广播**（broadcasting），附录A将对此进行详细讲解。

DataFrame和Series之间的运算差不多也是如此：

In [72]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
   .....:                      columns=list('bde'),
   .....:                      index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [73]:
series = frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

默认情况下，DataFrame和Series之间的算术运算**会将Series的索引匹配**到DataFrame的列，然后沿着行一直向下广播：

In [74]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集：【有差异就会进行**重新索引形成并集**】

In [75]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame+series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


如果你希望匹配行且在列上广播，则必须使用算术运算方法

传入的轴号就是希望**匹配的轴**。在本例中，我们的目的是匹配DataFrame的行索引（axis='index' or axis=0）并进行广播【沿列广播】。

In [76]:
series3 = frame['d']
frame.sub(series3, axis='index')

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


# 函数应用和映射

NumPy的ufuncs（元素级数组方法）也可用于操作pandas对象：

In [77]:
 frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
   .....:                      index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.544350,0.462547,-0.107305
Ohio,-1.095745,-1.203843,1.232504
Texas,-0.112594,-0.301789,0.513939
Oregon,2.008090,0.540167,-0.564037


In [78]:
np.abs(frame)

,b,d,e
Utah,0.544350,0.462547,0.107305
Ohio,1.095745,1.203843,1.232504
Texas,0.112594,0.301789,0.513939
Oregon,2.008090,0.540167,0.564037


另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。DataFrame的**apply方法**即可实现此功能：

In [79]:
f = lambda x : x.max()-x.min()
frame.apply(f)

b    3.103835
d    1.744010
e    1.796541
dtype: float64

这里的函数f，计算了一个Series的最大值和最小值的差，在frame的每列都执行了一次。

结果是一个Series，使用frame的列作为索引。

如果传递axis='columns'到apply【指定操作匹配的轴是列】，这个函数会在每行执行：

In [80]:
frame.apply(f, axis='columns')

Utah      0.651655
Ohio      2.436347
Texas     0.815728
Oregon    2.572127
dtype: float64

传递到apply的函数不是必须返回一个标量，还可以返回由多个值组成的Series：

In [81]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,-1.095745,-1.203843,-0.564037
max,2.008090,0.540167,1.232504


元素级的Python函数也是可以用的。假如你想得到frame中各个浮点值的格式化字符串，使用**applymap方法**即可：

In [82]:
format = lambda x: '%.2f' % x
frame.applymap(format)

,b,d,e
Utah,0.54,0.46,-0.11
Ohio,-1.10,-1.20,1.23
Texas,-0.11,-0.30,0.51
Oregon,2.01,0.54,-0.56


之所以叫做applymap，是因为Series有一个用于应用元素级函数的**map方法**：

In [83]:
frame['e'].map(format)

Utah      -0.11
Ohio       1.23
Texas      0.51
Oregon    -0.56
Name: e, dtype: object

# 排序和排名

根据条件对数据集排序（sorting）也是一种重要的内置运算。

要对行或列索引进行排序（按字典顺序），可使用**sort_index方法**，它将返回一个已排序的新对象：

In [84]:
obj = pd.Series(range(4), index=['d','a','b','c'])
obj

d    0
a    1
b    2
c    3
dtype: int32

In [85]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int32

对于DataFrame，则可以根据任意一个轴上的索引进行排序：

In [86]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
   .....:                      index=['three', 'one'],
   .....:                      columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [87]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [88]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


若要按值对Series进行排序，可使用其**sort_values方法**：

In [89]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

在排序时，任何缺失值默认都会被放到Series的末尾：

In [90]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

当排序一个DataFrame时，你可能希望根据一个或多个列中的值进行排序。

将一个或多个列的名字传递给sort_values的**by参数选项**即可达到该目的：

In [91]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,a,b
0,0,4
1,1,7
2,0,-3
3,1,2


In [92]:
frame.sort_values(by='b')

,a,b
2,0,-3
3,1,2
0,0,4
1,1,7


要根据多个列进行排序，传入名称的列表即可：

In [93]:
frame.sort_values(by=['a', 'b'])

,a,b
2,0,-3
0,0,4
3,1,2
1,1,7


接下来介绍Series和DataFrame的**rank方法**。默认情况下，rank是通过“为各组分配一个平均排名”的方式破坏平级关系的：

In [94]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

也可以根据值在原数据中出现的顺序给出排名：设定**method参数选项**

这里，条目0和2没有使用平均排名6.5，它们被设成了6和7，因为数据中标签0位于标签2的前面。

In [95]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

你也可以按降序进行排名，设定ascending参数选项：

In [96]:
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

DataFrame可以在行或列上计算排名：

In [97]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
   .....:                       'c': [-2, 5, 8, -2.5]})
frame

,a,b,c
0,0,4.3,-2.0
1,1,7.0,5.0
2,0,-3.0,8.0
3,1,2.0,-2.5


In [98]:
frame.rank(axis='columns')

,a,b,c
0,2.0,3.0,1.0
1,1.0,3.0,2.0
2,2.0,1.0,3.0
3,2.0,3.0,1.0


> 表5-6列出了所有用于破坏平级关系的method选项。

> average默认值 | min | max | first | dense |

# 带有重复标签的轴索引

直到目前为止，我所介绍的所有范例都有着唯一的轴标签（索引值）。虽然许多pandas函数（如reindex）都要求标签唯一，但这并不是强制性的。

我们来看看下面这个简单的带有重复索引值的Series：

索引的is_unique属性可以告诉你它的值是否是唯一的：

In [99]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int32

In [100]:
obj.index.is_unique

False

对于带有重复值的索引，数据选取行为将会有些不同。如果某个索引对应多个值，则返回一个Series；而对应单个值的，则返回一个标量值：

这样会使代码变复杂，因为索引的输出类型会根据标签是否有重复发生变化。

In [101]:
obj['a']

a    0
a    1
dtype: int32

In [102]:
obj['c']

4

对DataFrame的行进行索引时也是如此：

In [103]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df.loc['b']

,0,1,2
b,-1.659180,0.503501,0.476401
b,-1.214599,-1.474904,0.181918


# 5.3 汇总和计算描述统计-----------------------------------------------------------------------------

pandas对象拥有一组常用的数学和统计方法。它们大部分都属于约简和汇总统计，用于从Series中提取单个值（如sum或mean）或从DataFrame的行或列中提取一个Series。跟对应的NumPy数组方法相比，它们都是基于没有缺失数据的假设而构建的。看一个简单的DataFrame：

In [104]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
   .....:                    [np.nan, np.nan], [0.75, -1.3]],
   .....:                   index=['a', 'b', 'c', 'd'],
   .....:                   columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


调用DataFrame的**sum方法**将会返回一个含有列的和的Series：【运算操作的轴是行】

In [105]:
df.sum()

one    9.25
two   -5.80
dtype: float64

传入axis='columns'或axis=1将会按行进行求和运算：【运算操作的轴是列】

In [106]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

NA值会自动被排除，除非整个切片（这里指的是行或列）都是NA。通过**skipna选项**可以禁用该功能：

In [107]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

有些方法（如idxmin和idxmax）返回的是间接统计（比如达到最小值或最大值的索引）：

另一些方法则是累计型的cumsum

还有一种方法，它既不是约简型也不是累计型。describe就是一个例子，它用于一次性产生多个汇总统计

> 表5-8列出了所有与描述统计相关的方法。

> count | describe | min max | argmin argmax idxmin idxmax |

> quantile分位数 | sum men median |

> mad var std skew kurt |

> cumsum |cummin cummax cumprod | diff | pct_change百分数变化

### 相关系数与协方差

有些汇总统计（如相关系数和协方差）是通过参数对计算出来的。我们来看几个DataFrame

Series的**corr方法**用于计算两个Series中重叠的、非NA的、按索引对齐的值的相关系数。与此类似，**cov方法**用于计算协方差：

### 唯一值、值计数以及成员资格

还有一类方法可以从一维Series的值中抽取信息。看下面的例子：

第一个函数是**unique方法**，它可以得到Series中的唯一值数组：

In [108]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

返回的唯一值是未排序的，如果需要的话，可以对结果再次进行排序（uniques.sort()）。

相似的，**value_counts方法**用于计算一个Series中各值出现的频率：

为了便于查看，结果Series是按值频率降序排列的。value_counts还是一个顶级pandas方法，可用于任何数组或序列：

In [109]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

**isin方法**用于判断矢量化集合的成员资格，可用于过滤Series中或DataFrame列中数据的子集：

In [110]:
mask = obj.isin(['b','c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

与isin类似的是**Index.get_indexer方法**，它可以给你一个索引数组，从可能包含重复值的数组到另一个不同值的数组：

In [111]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c','b','a'])
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2], dtype=int32)

有时，你可能希望得到DataFrame中多个相关列的一张柱状图。例如：

将pandas.value_counts传给该DataFrame的apply函数，就会出现：

In [112]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
   .....:                      'Qu2': [2, 3, 1, 2, 3],
   .....:                      'Qu3': [1, 5, 2, 4, 4]})
result = data.apply(pd.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
